In [ ]:
%run superuser-init.py --overwrite true

## Set up instance

In [ ]:
import openreview
import csv
import re
import requests
from collections import defaultdict

client = openreview.Client(username="OpenReview.net",password="OpenReview_beta",baseurl="http://localhost:3000")

def assign_reviewer(reviewer,paper_number,submissions):
    notes = [note for note in submissions if str(note.number)==str(paper_number)]
    
    valid_email = re.compile('^[^@\s,]+@[^@\s,]+\.[^@\s,]+$')
    valid_tilde = re.compile('~.+')
    reviewer_group = None
    if not notes:
        print "Paper number " + paper_number + " does not exist"
    elif not valid_email.match(reviewer) and not valid_tilde.match(reviewer):
        print "Program Committee Member \""+reviewer+"\" invalid. Please check for typos and whitespace."
    else:
        #need to incorporate conflicts. get them from public profile? confirm this with UAI documentation
        reviewer_group = get_reviewer_group(reviewer, paper_number, [])
    return reviewer_group

def get_reviewer_group(reviewer, paper_number, conflict_list):

    reviewers = client.get_group('auai.org/UAI/2017/Paper'+paper_number+'/Reviewers')
    nonreaders_reviewers = client.get_group('auai.org/UAI/2017/Paper'+paper_number+'/Reviewers/NonReaders')
    existing_reviewers = reviewers.members

    conference_reviewers = client.get_group(PC)
    if not (reviewer in conference_reviewers.members):
        client.add_members_to_group(conference_reviewers,reviewer)

    next_reviewer = get_next_reviewer_id(reviewer, paper_number)

    new_reviewer_id = 'auai.org/UAI/2017/Paper' + str(paper_number) + '/' + next_reviewer
    new_reviewer = create_reviewer_group(new_reviewer_id, reviewer, paper_number, conflict_list)

    client.add_members_to_group(reviewers,[reviewer])
    client.add_members_to_group(nonreaders_reviewers,[new_reviewer.id])
    
    return new_reviewer


def get_next_reviewer_id(reviewer, paper_number):

    headers = {
    'User-Agent': 'test-create-script',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + client.token
    }

    response = requests.get(client.baseurl + '/groups?id=auai.org/UAI/2017/Paper' + paper_number + '/AnonReviewer.*', headers=headers)
    reviewers = response.json()

    if reviewers:
        numbers = []
        for existing_reviewer in reviewers:

            if reviewer in existing_reviewer['members']:
                print "reviewer " + reviewer + " found in " + existing_reviewer['id']
                return None
            numbers.append(int(existing_reviewer['id'].split('AnonReviewer')[1]))

        return "AnonReviewer" + str(max(numbers) + 1)

    else:
        return "AnonReviewer1"




def create_reviewer_group(new_reviewer_id, reviewer, paper_number, conflict_list):
    new_reviewer = openreview.Group(
        new_reviewer_id,
        signatures=['auai.org/UAI/2017'],
        writers=['auai.org/UAI/2017'],
        members=[reviewer],
        readers=['auai.org/UAI/2017', COCHAIRS, SPC, PC],
        nonreaders=conflict_list,
        signatories=[new_reviewer_id])
    client.post_group(new_reviewer)
    return new_reviewer

### Generate dummy reviewers

In [ ]:
last = 'Reviewer'
password = '1234'

users = []
for first in ['Red', 'Orange', 'Yellow', 'Green', 'Blue', 'Indigo', 'Violet']:
    print first+" Reviewer"
    email = ('%sreviewer@gmail.com' % first.lower())
    users.append(client.create_dummy(email=email,first=first,last=last,password=password,secure_activation=False))

reviewers = [u['user']['profile']['id'] for u in users]

### Generate fake paper submissions

In [ ]:
#modify the submission invitation so that we don't have to submit a pdf
uaisub = client.get_invitation('auai.org/UAI/2017/-/submission')
uaisub.reply['content']['pdf']['value-regex'] = '.*'
client.post_invitation(uaisub)

def get_content(color):
    
    abstracts = {
        'Red': '''Red is the color at the longer-wavelengths end of the spectrum of visible light next to orange, at the opposite end from violet.[3] Red color has a predominant light wavelength of roughly 620–740 nanometers. Light with a longer wavelength than red but shorter than terahertz radiation and microwave is called infrared.
Red is one of the additive primary colors of visible light, along with green and blue, which in Red Green Blue (RGB) color systems are combined to create all the colors on a computer monitor or television screen. Red is one of the subtractive secondary colors, resulting from the combination of yellow and magenta. (See CMYK color model.) Traditionally, it was viewed as a primary subtractive colour, along with yellow and blue, in the RYB color space and traditional color wheel formerly used by painters and artists. Reds can vary in shade from very light pink to very dark maroon or burgundy; and in hue from the bright orange-red scarlet or vermilion to the bluish-red crimson. Red is the complementary color of cyan.''',
        'Orange':'''Orange is the colour between red and yellow in the spectrum of light, and on the traditional colour wheel used by painters. Its name is derived from the fruit orange.
In Europe and America, surveys show that orange is the colour most associated with amusement, the unconventional, extroverts, warmth, fire, energy, activity, danger, taste and aroma, the autumn season, and Protestantism, as well as having long been the national colour of the Netherlands and the House of Orange. In Asia it is an important symbolic colour of Buddhism and Hinduism.[1]''',
        'Yellow':'''Yellow is the color between green and orange in the spectrum of visible light. It is the color of ripe lemons, sunflowers, and gold. It is a primary color in subtractive color, used in color printing. According to surveys in Europe, Canada, and the United States, yellow is the color people most often associate with amusement, gentleness, and spontaneity, but also with duplicity, envy, jealousy, avarice, and, in the U.S., with cowardice. It plays an important role in Asian culture, particularly in China, where it is seen as the color of happiness, glory, wisdom, harmony, and culture.[3]''',
        'Green':'''Green is the color between blue and yellow on the spectrum of visible light. It is evoked by light with a predominant wavelength of roughly 495–570 nm. In the subtractive color system, used in painting and color printing, it is created by a combination of yellow and blue, or yellow and cyan; in the RGB color model, used on television and computer screens, it is one of the additive primary colors, along with red and blue, which are mixed in different combinations to create all other colors.
The modern English word green comes from the Middle English and Anglo-Saxon word grene, from the same Germanic root as the words "grass" and "grow".[1] It is the color of living grass and leaves[2][3] and as a result is the color most associated with springtime, growth and nature.[4] By far the largest contributor to green in nature is chlorophyll, the chemical by which plants photosynthesize and convert sunlight into chemical energy. Many creatures have adapted to their green environments by taking on a green hue themselves as camouflage. Several minerals have a green color, including the emerald, which is colored green by its chromium content.''',
        'Blue':'''Blue is the colour between violet and green on the optical spectrum of visible light. Human eyes perceive blue when observing light with a wavelength between 450 and 495 nanometres, which is between 4500 and 4950 ångströms. Blues with a higher frequency and thus a shorter wavelength gradually look more violet, while those with a lower frequency and a longer wavelength gradually appear more green. Pure blue, in the middle, has a wavelength of 470 nanometers (4700 ångströms). In painting and traditional colour theory, blue is one of the three primary colours of pigments, along with red and yellow, which can be mixed to form a wide gamut of colours. Red and blue mixed together form violet, blue and yellow together form green. Blue is also a primary colour in the RGB colour model, used to create all the colours on the screen of a television or computer monitor.''',
        'Indigo': '''Indigo is a color that is traditionally regarded as a color in the visible spectrum, as well as one of the seven colors of the rainbow: the color between blue and violet. Although traditionally considered one of seven major spectral colors, sources differ as to its actual position in the electromagnetic spectrum. Indigo is a deep and bright color close to the color wheel blue (a primary color in the RGB color space), as well as to some variants of ultramarine.
The color indigo is named after the indigo dye derived from the plant Indigofera tinctoria and related species.
The first known recorded use of indigo as a color name in English was in 1289.[3]'
''',
        'Violet':'''Violet is the color of amethyst, lavender and beautyberries. It takes its name from the violet flower.[2][3]
Violet is the color at the end of the visible spectrum of light between blue and the invisible ultraviolet. Violet color has a predominant light wavelength of roughly 380-450 nanometers[4] (in experiments under special conditions, people have so far seen to 310 nm).[5][6][7] Light with a shorter wavelength than violet but longer than X-rays and gamma rays is called ultraviolet. In the color wheel historically used by painters, it is located between blue and purple. On the screens of computer monitors and television sets, a color which looks similar to violet is made, with the RGB color model, by mixing red and blue light, with the blue twice as bright as the red. This is not true violet, since it is composed of multiple longer wavelengths rather than a single wavelength shorter than that of blue light.'''
    }
    
    generic_content = {
        'pdf': '%s.pdf' % color,
        'title': '%s Paper' % color,
        'abstract': abstracts[color],
        'authors': 'Michael Spector',
        'keywords':['%s Keyword' % color],
        'TL;DR':'',
        'authorids':['spector@cs.umass.edu'],
        'student paper':'Yes',
        'subject areas':['Algorithms: Approximate Inference']
    }
    return generic_content


#Create three sample notes
sample_papers = [None]*7

red_content = get_content('Red')
sample_papers[0] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content=red_content,
                                readers=['auai.org/UAI/2017'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )

orange_content = get_content('Orange')
sample_papers[1] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content=orange_content,
                                readers=['auai.org/UAI/2017'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )

yellow_content = get_content('Yellow')
sample_papers[2] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content=yellow_content,
                                readers=['auai.org/UAI/2017'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )

green_content = get_content('Green')
sample_papers[3] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content=green_content,
                                readers=['auai.org/UAI/2017'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )

blue_content = get_content('Blue')
sample_papers[4] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content=blue_content,
                                readers=['auai.org/UAI/2017'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )

indigo_content = get_content('Indigo')
sample_papers[5] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content=indigo_content,
                                readers=['auai.org/UAI/2017'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )

violet_content = get_content('Violet')
sample_papers[6] = openreview.Note(invitation='auai.org/UAI/2017/-/submission',
                                content=violet_content,
                                readers=['auai.org/UAI/2017'],
                                signatures=['~Michael_Spector1'],
                                writers=['~Michael_Spector1']
                              )

for note in sample_papers:
    print note.content['title']
    posted_note = client.post_note(note) 

In [ ]:
program_committee = client.get_group('auai.org/UAI/2017/Program_Committee')
program_committee.members = reviewers
client.post_group(program_committee)
print "added reviewers to Program Committee"

## Matching Demo

In [ ]:
blind_submissions = client.get_notes(invitation='auai.org/UAI/2017/-/blind-submission')
titlesdict = {note.forum: note.content['title'] for note in blind_submissions}

In [ ]:
%run populate_papers_data.py --betas ../dummy-data/paper-betas.xml --reviewerscores ../dummy-data/paper-reviewer-scores.xml --paperscores ../dummy-data/paper-paper-scores.xml --bidscores ../dummy-data/bids-by-papers.xml
%run populate_reviewer_data.py --alphas ../dummy-data/reviewer-alphas.xml --reviewerscores ../dummy-data/reviewer-reviewer-scores.xml

In [ ]:
%run populate_paper_reviewer_assignment.py 

In [ ]:
assignments = [(paper_forum_reviewers_dict[forum][0],paper_forum_number_dict[forum]) for forum in titlesdict.keys()]
assignments

In [ ]:
for reviewer, papernumber in assignments:
    new_reviewer = assign_reviewer(reviewer,str(papernumber),blind_submissions)
    print "Assigning %s to %s" % (reviewer,new_reviewer.id)


In [ ]:
pc = client.get_group('auai.org/UAI/2017/Program_Co-Chairs')
client.add_members_to_group(pc,['spector@cs.umass.edu','zach.bialecki@gmail.com','mbok@cs.umass.edu'])

In [ ]:
#client.create_dummy(first="UAI",last="Admin", email="uai2017admin", password="12345678", secure_activation=False)